In [3]:
## imports .
#1
#if you need to install any dependences
#run the cells at the end of this jupyter notebook
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pickle
from joblib import dump, load
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
pd.options.mode.chained_assignment = None

In [4]:
print("version 09321")

version 09321


In [28]:
##required functions
#run this cell second
#2

# get numpy arrays for training models
# for X, and y cations , y anions
def getXY(df):
  Xs = df[["hlr2","hlr3","hlr4","hlr5","hlr6","hlr7","hlr8"]]
  ys = df[["CationC"]]
  yz = df[["AnionC"]]
  return Xs.to_numpy(), ys.to_numpy(), yz.to_numpy()

# get features from the models database
def models_database(direction):
    database = pd.read_csv(direction)
    database = database[['hlr2','hlr3','hlr4','hlr5','hlr6','hlr7','hlr8',"CationC","AnionC"]]
    return database


def intercalate(a, b):
    c = np.array(0)
    for i in range(len(a)):
        c = np.append(c, a[i])
        c = np.append(c, b[i])
    return c

# Outputs extended report
def table_usr(models_database, cation_models, anion_models, name_aux): # this would define the desicions as the original metrics function
    # getting database info for the models.
    X_train = models_database[["hlr2","hlr3","hlr4", "hlr5","hlr6","hlr7","hlr8"]]
    mgL = models_database[["SampleID","Ca (mg/L)","Mg (mg/L)", "Na (mg/L)", "K (mg/L)",
                        "SO4 (mg/L)", "Cl (mg/L)", "HCO3 (mg/L)", "CO3 (mg/L)",
                        "Ca_mmol","Mg_mmol","Na_mmol","K_mmol",
                        "SO4_mmol","Cl_mmol","HCO3_mmol","CO3_mmol","GMC",
                        "hlr2","hlr3","hlr4", "hlr5","hlr6","hlr7","hlr8"]]

    for c, a, nom in zip(cation_models, anion_models, name_aux):
        print(" ")
        print("process in: ", c, a, "...")
        _model_c = load("models/"+c)# load cation model
        _model_a = load("models/"+a)# load anion model
        cation_arr, anion_arr = get_basic_hibrid(_model_c, _model_a, X_train)
        print("completed")
        print("")
        # concat cation_arr, anion_arr
        model_arr = np.concatenate((cation_arr, anion_arr), axis=1)
        model_pd = pd.DataFrame(model_arr)
        model_pd.columns=["Ca_p", "Mg_p", "Na_p", "K_p", "basic_c", "hibrid_c",
                         "SO4_p", "Cl_p", "HCO3_p", "CO3_p", "basic_a", "hibrid_a"]
        # setting the hibrid finals and basic
        model_pd[nom+""] = model_pd["basic_c"]+"-"+model_pd["basic_a"]
        model_pd[nom+" hybrid"] = model_pd["hibrid_c"]+"-"+model_pd["hibrid_a"]
        model_pd = model_pd.drop(columns=['basic_c', 'hibrid_c', 'basic_a', "hibrid_a"])
        mgL = pd.concat([mgL, model_pd], axis=1)
         
    
    mgL = mgL.rename(columns={"SampleID": "identification", "Ca_mmol":"Ca (mMol/L)",
                             "Mg_mmol": "Mg (mMol/L)", "Na_mmol":"Na (mMol/L)", "K_mmol": "K (mMol/L)",
                             "SO4_mmol": "SO4 (mMol/L)", "Cl_mmol":"Cl (mMol/L)", "HCO3_mmol":"HCO3 (mMol/L)"
                             ,"CO3_mmol":"CO3 (mMol/L)"}, errors="raise")    
    return mgL

    

def get_basic_hibrid(model_cation, model_anion, x):
    x = preprocessing.normalize(x, norm='l2')
    cat_prob = model_cation.predict_proba(x)
    an_prob = model_anion.predict_proba(x)
    cat_names = {1:"Ca", 2:"Mg", 3:"Na", 4:"K"}
    an_names = {1:"SO4", 2:"Cl", 3:"HCO3", 4:"CO3"}
    

    cat_probs = get_names(cat_prob, cat_names)
    an_probs = get_names(an_prob, an_names)
    
    return cat_probs, an_probs

def get_names(probs, names):
    
    #getting the basic name
    max1 = probs.argmax(axis=1) #this gives the index of the max value
    zp = np.copy(probs) # to copy the array
    for n in range(probs.shape[0]): #setting the max1 values to 0
        zp[n,max1[n]] = 0
        
    
    max2 = zp.argmax(axis=1)
    typp = np.zeros_like(max1)
    
    for i in range(probs.shape[0]):
        pm = probs[i, max1[i]]
        pn = probs[i, max2[i]]
        
        hib = True
        if(pm>=0.5 and (pm-pn)>=0.25 and pn<=0.25):
            hib = False
            
        max1[i]= max1[i]+1
        max2[i]= max2[i]+1
            
        if(hib==False):
            typp[i] = max1[i]
        else:
            typp[i] = max1[i]*10 + max2[i]
            
    str_hib = np.array(["zero"])
    str_bas = np.array(["zero"])
    
    for i in range(typp.shape[0]):
        if(typp[i]>9):
            nd = typp[i]%10
            st = (typp[i]-nd)/10
            str_hib = np.append(str_hib, names[st]+"-"+names[nd])
            str_bas = np.append(str_bas, names[max1[i]])
        else:
            # just one digit
            str_hib = np.append(str_hib, names[max1[i]])
            str_bas = np.append(str_bas, names[max1[i]])

            
    # delete the first one
    str_hib = np.delete(str_hib, 0)
    str_bas = np.delete(str_bas, 0)

    end  = np.c_[probs, str_bas, str_hib] 
    return end

# filter extended report 
# then output short report
def table_usr_filter(dat):
    det = dat[["identification","Ca (mg/L)","Mg (mg/L)","Na (mg/L)","K (mg/L)",
               "SO4 (mg/L)","Cl (mg/L)","HCO3 (mg/L)","CO3 (mg/L)",
               "Ca (mMol/L)","Mg (mMol/L)","Na (mMol/L)","K (mMol/L)",
               "SO4 (mMol/L)","Cl (mMol/L)","HCO3 (mMol/L)","CO3 (mMol/L)", "GMC",
              "WClassCB","WClassCB hybrid",
               "WClassVL","WClassVL hybrid",
              "WClassVP","WClassVP hybrid",
              "WClassVR","WClassVR hybrid"]]
    return det



# model names which are required to use other methods
models_cat = np.array(["WClassCB_cation", 
              "WClassVL_cation",
              "WClassVP_cation", 
              "WClassVR_cation"])
models_an = np.array(["WClassCB_anion", 
             "WClassVL_anion", 
             "WClassVP_anion"
            ,"WClassVR_anion"])

# detemrines greater molar conc water types
def gmc(samples):
    cation = samples[["Ca_mmol","Mg_mmol","Na_mmol","K_mmol"]]
    anion = samples[["SO4_mmol","Cl_mmol","HCO3_mmol","CO3_mmol"]]
    tempp = samples[["Ca_mmol"]]
    
    cat_numbers = np.zeros_like(tempp)
    an_numbers = np.zeros_like(tempp)
    
    for i in range(len(cation)):
        cat = cation.iloc[i]
        an = anion.iloc[i]
        cat = cat.to_numpy()
        an = an.to_numpy()
        cat = np.argmax(cat)
        an = np.argmax(an)
        cat_numbers[i] = cat
        an_numbers[i] = an
        
    cats = pd.Series(cat_numbers.ravel())
    ans = pd.Series(an_numbers.ravel())
    
    cats = cats.map({0: "Ca", 1: "Mg", 2:"Na", 3:"K"})
    ans = ans.map({0:"SO4",1:"Cl",2:"CO3",3:"HCO3"})
    
    joint = cats+"-"+ans
    samples["GMC"] = joint
        
# main function to classify water samples
def classify_samples(data):
    print("classifying samples")
    database = mg_to_hlr(data)
    database = database.dropna()
    gmc(database)
    filename = data.replace(".csv", "")
    model_column_names = ["WClassCB","WClassVL","WClassVP","WClassVR"]
    f = table_usr(database, models_cat, models_an, model_column_names)
    f.to_csv("extended_report_"+filename+".csv")
    ff =table_usr_filter(f)
    ff.to_csv("short_report_"+filename+".csv")
    print("A total of",len(database),"samples have been classificated")
    ### fix column names

# calculate hlr transformations
def mg_to_hlr(data):
    complete = pd.read_csv(data)
    complete = complete[['SampleID','Ca (mg/L)','Mg (mg/L)','Na (mg/L)','K (mg/L)',
                         'SO4 (mg/L)','Cl (mg/L)','HCO3 (mg/L)',"CO3 (mg/L)"]]
    
    complete["Ca (mmol/L)"] = complete["Ca (mg/L)"]
    complete[["Ca_mmol"]] = complete[["Ca (mg/L)"]] / 40.078
    complete[["Mg_mmol"]] = complete[["Mg (mg/L)"]]/ 24.305
    complete[["Na_mmol"]] =  complete[["Na (mg/L)"]]  / 22.98976928
    complete[["K_mmol"]] = complete[["K (mg/L)"]] / 39.0983
    complete[["SO4_mmol"]] = complete[["SO4 (mg/L)"]] / 96.056
    complete[["Cl_mmol"]] = complete[["Cl (mg/L)"]] / 35.45
    complete[["HCO3_mmol"]]  =complete[["HCO3 (mg/L)"]] / 61.016
    complete[["CO3_mmol"]] =complete[["CO3 (mg/L)"]] / 60.008

    Camol, Mgmol = complete[["Ca_mmol"]].to_numpy(), complete[["Mg_mmol"]].to_numpy()
    Namol, Kmol = complete[["Na_mmol"]].to_numpy(), complete[["K_mmol"]].to_numpy()
    SO4mol, Clmol = complete[["SO4_mmol"]].to_numpy(), complete[["Cl_mmol"]].to_numpy()
    HCO3mol, CO3mol = complete[["HCO3_mmol"]].to_numpy(), complete[["CO3_mmol"]].to_numpy()

    #calculate hlr
    complete[["hlr2"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / Mgmol)
    complete[["hlr3"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / Namol)
    complete[["hlr4"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / Kmol)
    complete[["hlr5"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / SO4mol)
    complete[["hlr6"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / Clmol)
    complete[["hlr7"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / HCO3mol)
    complete[["hlr8"]] = np.log(np.power(Camol*Mgmol*Namol*Kmol*SO4mol*Clmol*HCO3mol*CO3mol, 1/8) / CO3mol)

    return complete
    
    

In [29]:
# RUN THIS CELL TO USE THE MODELS
#    sampels with null values will be droped and not considered in the
#    use template.csv to set up new water samples 
classify_samples("template_water.csv")

classifying samples
 
process in:  WClassCB_cation WClassCB_anion ...
completed

 
process in:  WClassVL_cation WClassVL_anion ...
completed

 
process in:  WClassVP_cation WClassVP_anion ...
completed

 
process in:  WClassVR_cation WClassVR_anion ...
completed

A total of 16 samples have been classificated


In [30]:
# RUN THIS TO SEE PARAMETERS OF MACHINE LEARNING MODELS
# [list of models]
#Catboost
#SVM with linear kernel
#SVM with polynomial kernel
#SVM with radial basis function kernel

cb_cat = load("models/WClassCB_cation")
cb_an = load("models/WClassCB_anion")
linear_cat = load("models/WClassVL_cation")
linear_an = load("models/WClassVL_anion")
poly_cat = load("models/WClassVP_cation")
poly_an = load("models/WClassVP_anion")
rbf_cat = load("models/WClassVR_cation")
rbf_an = load("models/WClassVR_anion")
print("*** [WClassCB] catboost cation")
print(cb_cat.get_params())
print("*** [WClassCB] catboost anion")
print(cb_an.get_params())
print("")
print("*** [WClassVL] svm with linear kernel cation")
print(linear_cat.get_params())
print("*** [WClassVL] svm with linear kernel anion")
print(linear_an.get_params())
print("")
print("*** [WClassVP] svm with polynomial kernel cation")
print(poly_cat.get_params())
print("*** [WClassVP] svm with polynomial kernel anion")
print("")
print(poly_an.get_params())
print("*** [WClassVR] svm with radial kernel cation")
print(rbf_cat.get_params())
print("*** [WClassVR] svm with radial kernel anion")
print(rbf_an.get_params())

*** [WClassCB] catboost cation
{'estimator__iterations': 2000, 'estimator__learning_rate': 0.1, 'estimator__depth': 4, 'estimator__random_state': 42, 'estimator': <catboost.core.CatBoostClassifier object at 0x000002063F4F0280>, 'n_jobs': None}
*** [WClassCB] catboost anion
{'estimator__iterations': 2000, 'estimator__learning_rate': 0.1, 'estimator__depth': 4, 'estimator__random_state': 42, 'estimator': <catboost.core.CatBoostClassifier object at 0x000002067093C070>, 'n_jobs': None}

*** [WClassVL] svm with linear kernel cation
{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'probability': True, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}
*** [WClassVL] svm with linear kernel anion
{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 's

In [ ]:
# run the following cells to install dependences
pip install -U scikit-learn

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install catboost

In [ ]:
pip install pickle-mixin